# Pipeline Simulado - ALERJ 2025

Instalação das libs.

In [ ]:
%%capture captured_output
# Instala de forma silenciosa. Se der erro, avisa depois.
!pip install pdfplumber tqdm google-generativeai python-dotenv

In [ ]:
import sys

# Verifica se houve erro na saída capturada
if "ERROR:" in str(captured_output):
    print("❌ Ocorreu um erro na instalação:")
    captured_output.show()
else:
    print("✅ Todas as bibliotecas foram instaladas e estão prontas para uso.")

## Célula 1: Configuração e Bibliotecas

In [ ]:
import pdfplumber
import re
import json
import os
from pathlib import Path

# Configuração de Pastas (Cria automático se não existir)
PASTA_PDF = Path("pdf")     # Onde você joga os arquivos originais
PASTA_TXT = Path("txt")     # Onde fica o texto intermediário
PASTA_JSON = Path("json")   # Onde sai o resultado final

PASTA_TXT.mkdir(exist_ok=True)
PASTA_JSON.mkdir(exist_ok=True)

print("✅ Ambiente configurado. Pastas prontas.")

## Célula 2: Função de Extração (PDF → TXT)
#### Aqui usamos layout=False para imitar o comportamento de "copiar texto puro".

In [ ]:
def pdf_para_txt_stream(caminho_pdf):
    texto_completo = []
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            for page in pdf.pages:
                # layout=False ignora posições exatas e pega o fluxo de texto (melhor para sua prova)
                txt = page.extract_text(layout=False)
                if txt:
                    texto_completo.append(txt)
        return "\n".join(texto_completo)
    except Exception as e:
        print(f"❌ Erro no arquivo {caminho_pdf}: {e}")
        return None

## Célula 3: Função de Transformação (TXT → JSON)
#### A lógica blindada agora empacotada.

In [ ]:
def processar_texto_para_json(texto_completo, id_inicial=1):
    # 1. Extrair Gabarito
    gabarito_map = {}
    match_inicio = re.search(r'Respostas:', texto_completo, re.IGNORECASE)
    texto_fim = texto_completo[match_inicio.start():] if match_inicio else texto_completo[-5000:]
    
    matches_gab = re.findall(r'(\d+)\s+([A-E])', texto_fim)
    for num, letra in matches_gab:
        gabarito_map[int(num)] = letra

    # 2. Separar Questões
    divisao_gabarito = re.split(r'Respostas:', texto_completo, flags=re.IGNORECASE)
    texto_questoes = divisao_gabarito[0] if divisao_gabarito else texto_completo
    blocos = re.split(r'(Questão\s+\d+)', texto_questoes, flags=re.IGNORECASE)

    lista_final = []
    
    for i in range(1, len(blocos)-1, 2):
        cabecalho = blocos[i]
        conteudo = blocos[i+1]
        
        # ID Seguro
        match_num = re.search(r'\d+', cabecalho)
        if not match_num: continue
        id_q = int(match_num.group())

        # Processar Linhas (Desembaralhar alternativas)
        enunciado_parts = []
        opcoes_map = {}
        
        for linha in conteudo.split('\n'):
            # Limpeza rápida
            linha = linha.strip()
            if not linha or re.match(r'^\d{8,}$', linha) or "Acessar Lista" in linha: continue
            
            # Captura Alternativa (A, B, C...)
            match_alt = re.match(r'^([A-E])[\s\.\)\-](.*)', linha)
            if match_alt:
                opcoes_map[match_alt.group(1)] = match_alt.group(2).strip()
            elif len(linha) > 3:
                enunciado_parts.append(linha)

        # Montagem
        opcoes_lista = [opcoes_map.get(letra, "N/A") for letra in ['A','B','C','D','E']]
        letra_resp = gabarito_map.get(id_q)

        lista_final.append({
            "id": id_q,
            "text": " ".join(enunciado_parts),
            "options": opcoes_lista,
            "correct": letra_resp, # Sai "A", "B", etc.
            "explanation": ""
        })
        
    return lista_final

## Célula 4: O Executor (Pipeline)
#### Esta célula processa tudo de uma vez.

In [ ]:
# 1. Listar PDFs
arquivos = list(PASTA_PDF.glob("*.pdf"))
print(f"🚀 Iniciando Pipeline para {len(arquivos)} arquivos PDF...\n")

for arq_pdf in arquivos:
    print(f"📄 Processando: {arq_pdf.name}")
    
    # ETAPA 1: PDF -> TXT
    texto_cru = pdf_para_txt_stream(arq_pdf)
    
    if texto_cru:
        # Salva backup TXT (Opcional, mas boa prática)
        caminho_txt = PASTA_TXT / (arq_pdf.stem + ".txt")
        with open(caminho_txt, "w", encoding="utf-8") as f:
            f.write(texto_cru)
            
        # ETAPA 2: TXT -> JSON
        dados_json = processar_texto_para_json(texto_cru)
        
        # Salva JSON Final
        caminho_json = PASTA_JSON / (arq_pdf.stem + ".json")
        with open(caminho_json, "w", encoding="utf-8") as f:
            json.dump(dados_json, f, indent=2, ensure_ascii=False)
            
        print(f"   ✅ Gerado JSON com {len(dados_json)} questões.")
    else:
        print("   ❌ Falha na leitura do PDF.")

print("\n🏁 Pipeline Finalizado!")

In [ ]:
import sys

# Verifica se houve erro na saída capturada
if "ERROR:" in str(captured_output):
    print("❌ Ocorreu um erro na instalação:")
    captured_output.show()
else:
    print("✅ Todas as bibliotecas foram instaladas e estão prontas para uso.")

## Célula 5: O Executor (Pipeline)
#### Esta célula processa tudo de uma vez.

Passo 1: Criar o Banco e Importar (JSON → SQL)

In [ ]:
import sqlite3
import json
import os
from pathlib import Path

# Configuração
PASTA_JSON = Path("json")
DB_NAME = "simulado.db"

def criar_tabela(conn):
    cursor = conn.cursor()
    # Tabela robusta com campo de controle 'processado'
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS questoes (
            id_global INTEGER PRIMARY KEY AUTOINCREMENT,
            id_original INTEGER,
            arquivo_origem TEXT,
            enunciado TEXT,
            opcao_a TEXT,
            opcao_b TEXT,
            opcao_c TEXT,
            opcao_d TEXT,
            opcao_e TEXT,
            gabarito TEXT,
            explicacao TEXT,
            processado BOOLEAN DEFAULT 0
        )
    """)
    conn.commit()

def importar_json(conn, arquivo_json):
    cursor = conn.cursor()
    arquivo_nome = arquivo_json.name
    
    with open(arquivo_json, 'r', encoding='utf-8') as f:
        dados = json.load(f)
        
    cont = 0
    for q in dados:
        # Verifica se já existe para não duplicar (Idempotência)
        cursor.execute("SELECT 1 FROM questoes WHERE id_original = ? AND arquivo_origem = ?", 
                       (q['id'], arquivo_nome))
        if cursor.fetchone():
            continue
            
        # Insere
        cursor.execute("""
            INSERT INTO questoes (id_original, arquivo_origem, enunciado, 
                                  opcao_a, opcao_b, opcao_c, opcao_d, opcao_e, gabarito)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            q['id'], 
            arquivo_nome, 
            q['text'],
            q['options'][0], q['options'][1], q['options'][2], q['options'][3], q['options'][4],
            q['correct']
        ))
        cont += 1
    
    conn.commit()
    print(f"   -> Importadas {cont} questões de {arquivo_nome}")

def main():
    print("🗄️  INICIANDO SETUP DO BANCO DE DADOS...")
    
    conn = sqlite3.connect(DB_NAME)
    criar_tabela(conn)
    
    arquivos = list(PASTA_JSON.glob("*.json"))
    if not arquivos:
        print("❌ Nenhum JSON encontrado na pasta json/.")
        return

    for arq in arquivos:
        importar_json(conn, arq)
        
    print(f"\n✅ Banco '{DB_NAME}' pronto e populado!")
    conn.close()

if __name__ == "__main__":
    main()

Passo 2: O Worker de Enriquecimento (SQL + IA)

In [ ]:
import sqlite3
import google.generativeai as genai
import time
import os
from dotenv import load_dotenv

# Carrega a API Key do arquivo .env (Segurança!)
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    raise ValueError("❌ ERRO: Chave GEMINI_API_KEY não encontrada no arquivo .env")

# Configura a IA
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash') # Modelo rápido e barato

DB_NAME = "simulado.db"

def gerar_explicacao_didatica(enunciado, gabarito, opcoes):
    """
    Consulta a IA para gerar uma explicação simples e sucinta.
    """
    # Prompt de Engenharia (Role + Contexto + Formato)
    prompt = f"""
    Você é um professor especialista em concursos públicos com didática excelente para iniciantes.
    
    Tarefa: Explique POR QUE a alternativa ({gabarito}) é a correta para a questão abaixo.
    
    Regras Obrigatórias:
    1. Seja SUCINTO (máximo 3 frases curtas).
    2. Use linguagem SIMPLES (evite juridiquês ou termos complexos sem explicar).
    3. Fale diretamente com o aluno.
    4. Se o gabarito for nulo ou inválido, responda apenas: "Gabarito não fornecido para análise."
    
    Questão: {enunciado}
    Alternativas:
    A) {opcoes[0]}
    B) {opcoes[1]}
    C) {opcoes[2]}
    D) {opcoes[3]}
    E) {opcoes[4]}
    
    Gabarito Oficial: {gabarito}
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"   ⚠️ Erro na API: {e}")
        return None

def worker_processar_fila():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    print("🤖 INICIANDO WORKER DE IA (Pressione Ctrl+C para parar)...")
    
    while True:
        # Busca 1 questão pendente (que ainda não tem explicação)
        # Ignora questões onde o gabarito é nulo (não dá pra explicar o vazio)
        cursor.execute("""
            SELECT id_global, enunciado, gabarito, opcao_a, opcao_b, opcao_c, opcao_d, opcao_e 
            FROM questoes 
            WHERE explicacao IS NULL AND gabarito IS NOT NULL
            LIMIT 1
        """)
        tarefa = cursor.fetchone()
        
        if not tarefa:
            print("✅ Todas as questões válidas foram processadas!")
            break
            
        id_global, enunciado, gabarito, *opcoes = tarefa
        
        print(f"   ⚡ Processando Questão ID {id_global} (Gabarito: {gabarito})...", end="", flush=True)
        
        # Chama a IA
        explicacao = gerar_explicacao_didatica(enunciado, gabarito, opcoes)
        
        if explicacao:
            # Salva no banco
            cursor.execute("""
                UPDATE questoes 
                SET explicacao = ?, processado = 1 
                WHERE id_global = ?
            """, (explicacao, id_global))
            conn.commit()
            print(" OK!")
        else:
            print(" FALHOU (API Error).")
            time.sleep(5) # Espera um pouco se der erro
            
        # Pausa de segurança para não tomar Block da API (Rate Limit)
        time.sleep(2) 

    conn.close()

if __name__ == "__main__":
    worker_processar_fila()